In [1]:
from deck import *
from game import *
from player import *
from phases import Phases
import random
import torch
import time
from decision_transformer.decision_transformer import DecisionTransformer

In [2]:
data = {
    "Silver": [],
    "Gold": []
}
# Tuple State, Legal_actions, action, reward

In [3]:

policies = [None,None]
wins=[0,0]
n_games = 10


def get_move(index, state, move_indexes=None, move_strings=None):
    assert(move_indexes != None or move_strings != None), "at least one list must not be None {} {}".format(move_indexes,move_strings)

    if (move_indexes != None):
        length = len(move_indexes)
    else:
        length = len(move_strings)

    if policies[index] == None or length == 1:
        if move_indexes != None:
            move = random.sample(move_indexes,1)[0]
        else:
            move = random.sample(move_strings,1)[0]
    else:
        assert (False), "TODO"
        pass
    return move

In [4]:
# Simulate n_names times
for i in range(n_games):
    print("Starting game %d" % (i + 1))
    # Create game
    #game = Game([Player(get_8ed_core_gold_deck(), name="Gold"), Player(get_8ed_core_silver_deck(), name="Silver")])
    game = Game([Player(get_8ed_core_gold_deck(), name="Player1", deck_name="Gold"), Player(get_8ed_core_gold_deck(), name="Player2", deck_name="Gold")])
    game.start_game()

    game_data = {
        game.players[0].name: [],
        game.players[1].name: []
    }
    while not game.is_over():
        # Get player, valid moves
        player = game.player_with_priority
        active_player_life = player.life
        other_player_life =game.players[1 - player.index]
        
        move_indexes, move_strings = game.get_moves()
        state = game.get_board_string()

        life_total_1 =  game.players[0].life
        life_total_2 = game.players[0].life
        # Skip COMBAT_DAMAGE_STEP_510_1c, no choices in damage assignment
        if game.current_phase_index == Phases.COMBAT_DAMAGE_STEP_510_1c:
            assert(len(move_indexes) == 1), "Should only be 1 value move"
            game.make_move(move_indexes[0],False)
            print(player = game.player_with_priority.name)
            
            continue

        # Get move, may have to unroll action
        if isinstance(move_strings, ActionUnroller):
            unroller = move_strings
            while not unroller.is_done():
                moves = unroller.get_legal_moves()
                move_string = get_move(player.index,state, move_indexes=None, move_strings=moves)

                # Record Move
                if len(moves) > 1:
                    game_data[player.name].append([state, moves, move_string])

                #print(move_string)
                state = unroller.register_move(move_string)
            # Apply unrolled actions to game
            unroller.make_move()
        else:
            #print(state)
            move = get_move(player.index, state, move_indexes, move_strings)
            move_string = move_strings[move_indexes.index(move)]

            # Record Move
            if len(move_indexes) > 1:
                game_data[player.name].append([state, move_strings, move_string])

            #print(move_string)
            game.make_move(move, False)
        #move = mcts.uct(game, itermax=5)

    if game.players[1].has_lost:
        wins[0] += 1
        reward_0 = 1
        reward_1 = -1
        #print("State", state)
    elif game.players[0].has_lost:
        wins[1] += 1
        reward_0 = -1
        reward_1 = 1
        #print("State", state)
    # Add reward info
    name_0 = game.players[0].name
    name_1 = game.players[1].name
    len_0 = len(game_data[name_0])
    len_1 = len(game_data[name_1])
    for i in range(max(len_0, len_1)):
        if i <= len_0 - 1:
            game_data[name_0][i].append(reward_0)
        if i <= len_1 - 1:
            game_data[name_1][i].append(reward_1)
    # Add game_data to overall data
    for player in game.players:
        data[player.deck_name].append(game_data[player.name])
    
print("player 0 wins: %i, player 1 wins: %i" % (wins[0], wins[1]))


Starting game 1
Starting game 2
Starting game 3
Starting game 4
Starting game 5
Starting game 6
Starting game 7
Starting game 8
Starting game 9
Starting game 10
player 0 wins: 6, player 1 wins: 4


In [7]:
data['Gold'][0]

[['\n        player-color$ Gold$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Rampant Growth$Mountain$Forest$Mountain$Mountain$Forest$Forest$\n        generic-debt$ 0$\n        manapool$ White$ 0$ Blue$ 0$ Black$ 0$ Red$ 0$ Green$ 0$ Colorless$ 0$\n        opponent-cards$ 7$\n        self-lands-untapped$ $\n        self-lands-tapped$ $\n        opponent-lands-untapped$ $\n        opponent-lands-tapped$ $\n        self-creatures-untapped$ $\n        self-creatures-tapped$ $\n        opponent-creatures-untapped$ $\n        opponent-creatures-tapped$ $\n        attackers-blockers$ None$\n        damage-order$ None$\n        ',
  ['Mountain', 'Forest', 'Mountain', 'Mountain', 'Forest', 'Forest', 'Pass'],
  'Forest',
  1],
 ['\n        player-color$ Gold$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Rampant Growth$Mountain$Forest$Mountain$Mountain$Forest$\n        ge

In [6]:
len(data['Silver'])

0

In [7]:
data['Gold'][0][0]

['\n        player-color$ Gold$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Forest$Lava Axe$Goblin Raider$Forest$Mountain$Lava Axe$Mountain$\n        generic-debt$ 0$\n        manapool$ White$ 0$ Blue$ 0$ Black$ 0$ Red$ 0$ Green$ 0$ Colorless$ 0$\n        opponent-cards$ 7$\n        self-lands-untapped$ $\n        self-lands-tapped$ $\n        opponent-lands-untapped$ $\n        opponent-lands-tapped$ $\n        self-creatures-untapped$ $\n        self-creatures-tapped$ $\n        opponent-creatures-untapped$ $\n        opponent-creatures-tapped$ $\n        attackers-blockers$ None$\n        damage-order$ None$\n        ',
 ['Forest', 'Forest', 'Mountain', 'Mountain', 'Pass'],
 'Forest',
 -1]

In [65]:
wins

[26, 24]

In [8]:
len(data['Gold'][0])

32

In [11]:
len(data['Gold'][0][0][0])

671

In [75]:
max_per_game = -1
max_state_len = -1
# Count total amount of transitions
total_transitions = 0
for deck_name, deck_data in data.items():
    for game_data in deck_data:
        game_len = len(game_data)
        max_per_game = max(max_per_game, game_len)
        max_state_len = max(max_state_len, max([len(transition[0]) for transition in game_data]))
        total_transitions += game_len
print(total_transitions)
print(max_per_game)
print(max_state_len)

157392
154
1274


In [18]:
len(data['Gold'])

100

In [76]:
flat_data = []
for deck_type, deck_data in data.items():
    for game in deck_data:
        flat_data.extend(game)

In [20]:
len(flat_data)

6906

In [78]:
gold_data = data['Gold']

In [77]:
flat_array = np.array(flat_data, dtype=object)

In [74]:
np.savez_compressed('./data/test_data', flat_array)

In [96]:
actions = set(flat_array[:, 2])

In [97]:
print(len(actions))

141


In [77]:
print(data['Gold'][0][0])

['\n        player-color$ Gold$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Volcanic Hammer$Mountain$Forest$Forest$Mountain$Volcanic Hammer$Grizzly Bears$\n        generic-debt$ 0$\n        manapool$ White$ 0$ Blue$ 0$ Black$ 0$ Red$ 0$ Green$ 0$ Colorless$ 0$\n        opponent-cards$ 7$\n        self-lands-untapped$ $\n        self-lands-tapped$ $\n        opponent-lands-untapped$ $\n        opponent-lands-tapped$ $\n        self-creatures-untapped$ $\n        self-creatures-tapped$ $\n        opponent-creatures-untapped$ $\n        opponent-creatures-tapped$ $\n        attackers-blockers$ None$\n        damage-order$ None$\n        ', ['Mountain', 'Forest', 'Forest', 'Mountain', 'Pass'], 'Mountain', -1]


In [79]:
import torchtext

In [80]:
def customer_tokenizer(text):
    return text.split("$")

In [81]:
def customer_tokenizer_whole(text):
    return [text]

In [82]:
import collections

In [98]:
counter_obj_actions = collections.Counter()

In [83]:
# for action in actions:
#     sub_actions = action.split("$")
#     counter_obj_actions.update(sub_actions)

In [99]:
for action in actions:
    counter_obj_actions.update([action])

In [100]:
counter_obj_actions

Counter({'tapped_ability_0_Forest': 1,
         'Enormous Baloth$0$block$Goblin Raider$1': 1,
         'Hill Giant$0$block$Hill Giant$0': 1,
         'Norwood Ranger$0$block$Grizzly Bears$0': 1,
         'Enormous Baloth$1$block$Grizzly Bears$1': 1,
         'Enormous Baloth$0$block$Enormous Baloth$1': 1,
         'Hill Giant$1$block$Enormous Baloth$0': 1,
         'Norwood Ranger$0$block$Enormous Baloth$0': 1,
         'Pass': 1,
         'Enormous Baloth$0$block$Ogre Taskmaster$0': 1,
         'Grizzly Bears$1$block$Grizzly Bears$0': 1,
         'Spined Wurm$0$block$Spined Wurm$0': 1,
         'Norwood Ranger$1$block$Enormous Baloth$0': 1,
         'Grizzly Bears$0$block$Enormous Baloth$1': 1,
         'Enormous Baloth$0$block$Grizzly Bears$1': 1,
         'Norwood Ranger$1$block$Norwood Ranger$0': 1,
         'Norwood Ranger$1$block$Spined Wurm$0': 1,
         'Grizzly Bears$1$block$Enormous Baloth$0': 1,
         'Hill Giant$1$block$Goblin Raider$0': 1,
         'Grizzly Bears$1$bl

In [101]:
vocab_actions = torchtext.vocab.Vocab(counter_obj_actions, min_freq=1, specials=["<unk>"])

In [102]:
for i in range(len(vocab_actions)):
    print(i, vocab_actions.itos[i])

0 <unk>
1 Enormous Baloth
2 Enormous Baloth$0
3 Enormous Baloth$0$block$Enormous Baloth$0
4 Enormous Baloth$0$block$Enormous Baloth$1
5 Enormous Baloth$0$block$Goblin Raider$0
6 Enormous Baloth$0$block$Goblin Raider$1
7 Enormous Baloth$0$block$Grizzly Bears$0
8 Enormous Baloth$0$block$Grizzly Bears$1
9 Enormous Baloth$0$block$Hill Giant$0
10 Enormous Baloth$0$block$Hill Giant$1
11 Enormous Baloth$0$block$Norwood Ranger$0
12 Enormous Baloth$0$block$Norwood Ranger$1
13 Enormous Baloth$0$block$Ogre Taskmaster$0
14 Enormous Baloth$0$block$Spined Wurm$0
15 Enormous Baloth$1
16 Enormous Baloth$1$block$Enormous Baloth$0
17 Enormous Baloth$1$block$Enormous Baloth$1
18 Enormous Baloth$1$block$Goblin Raider$0
19 Enormous Baloth$1$block$Goblin Raider$1
20 Enormous Baloth$1$block$Grizzly Bears$0
21 Enormous Baloth$1$block$Grizzly Bears$1
22 Enormous Baloth$1$block$Hill Giant$0
23 Enormous Baloth$1$block$Hill Giant$1
24 Enormous Baloth$1$block$Norwood Ranger$0
25 Enormous Baloth$1$block$Norwood Ran

In [103]:
counter_obj_states = collections.Counter()

In [104]:
def process_state(state):
    return state.replace(' ', '').replace('\n', '').split('$')

In [35]:
print(flat_data[0][0].replace(' ', '').replace('\n', '').split('$'))

['player-color', 'Gold', 'life', '20', 'opponent-life', '20', 'phase', 'Phases.MAIN_PHASE_PRE_COMBAT', 'hand', 'Forest', 'LavaAxe', 'GoblinRaider', 'Forest', 'Mountain', 'LavaAxe', 'Mountain', 'generic-debt', '0', 'manapool', 'White', '0', 'Blue', '0', 'Black', '0', 'Red', '0', 'Green', '0', 'Colorless', '0', 'opponent-cards', '7', 'self-lands-untapped', '', 'self-lands-tapped', '', 'opponent-lands-untapped', '', 'opponent-lands-tapped', '', 'self-creatures-untapped', '', 'self-creatures-tapped', '', 'opponent-creatures-untapped', '', 'opponent-creatures-tapped', '', 'attackers-blockers', 'None', 'damage-order', 'None', '']


In [105]:
max_state_token_len = -1
for transition in flat_data:
    state = process_state(transition[0])
    max_state_token_len = max(max_state_token_len, len(state))
    counter_obj_states.update(state)


In [37]:
max_state_token_len

97

In [106]:
vocab_states = torchtext.vocab.Vocab(counter_obj_states, min_freq=1, specials=["<unk>"])

In [94]:
type(vocab_states)

torchtext.vocab.Vocab

In [39]:
len(vocab_states)

77

In [40]:
for i in range(len(vocab_states)):
    print(i, vocab_states.itos[i])

0 <unk>
1 Mountain
2 Forest
3 0
4 
5 None
6 Black
7 Blue
8 Colorless
9 Green
10 Red
11 White
12 attackers-blockers
13 damage-order
14 generic-debt
15 hand
16 life
17 manapool
18 opponent-cards
19 opponent-creatures-tapped
20 opponent-creatures-untapped
21 opponent-lands-tapped
22 opponent-lands-untapped
23 opponent-life
24 phase
25 player-color
26 self-creatures-tapped
27 self-creatures-untapped
28 self-lands-tapped
29 self-lands-untapped
30 GoblinRaider
31 HillGiant
32 EnormousBaloth
33 GrizzlyBears
34 NorwoodRanger
35 Phases.MAIN_PHASE_PRE_COMBAT
36 LavaAxe
37 20
38 OgreTaskmaster
39 1
40 SpinedWurm
41 Gold
42 Silver
43 2
44 3
45 4
46 VolcanicHammer
47 6
48 5
49 7
50 StoneRain
51 Phases.DECLARE_ATTACKERS_STEP
52 RampantGrowth
53 17
54 18
55 8
56 10
57 15
58 11
59 attacker:
60 blockedby:
61 9
62 12
63 16
64 13
65 14
66 19
67 Phases.DECLARE_BLOCKERS_STEP
68 -3
69 -4
70 -5
71 Phases.DECLARE_BLOCKERS_STEP_509_2
72 -2
73 0']
74 1']
75 ['GoblinRaider
76 ['GrizzlyBears


In [107]:
trajectories = []

In [108]:
for game in data['Gold']:
    game_data = {'states': [], 'actions': [], 'rewards':[], 'timesteps':[], 'state_masks':[]}
    timestep = 0
    for trajectory in game:
        state = [vocab_states[word] for word in process_state(trajectory[0])]
        mask = [1] * len(state)
        padding_len = max_state_token_len - len(state)
        mask.extend([0] * padding_len)
        state.extend([0] * padding_len)

        game_data['state_masks'].append(mask)
        game_data['states'].append(state)
        game_data['actions'].append(vocab_actions[trajectory[2]])
        game_data['rewards'].append(trajectory[3])
        game_data['timesteps'].append(timestep)
        timestep +=1
    for key,value in game_data.items():
        if not isinstance(value, np.ndarray):
            game_data[key] = np.array(value)
    trajectories.append(game_data)

In [109]:
embed_dim=128
dropout=.1

hidden_size=embed_dim
n_layer=3
n_head=1
n_inner=4*embed_dim
activation_function='relu'
n_positions=1024
resid_pdrop=dropout
attn_pdrop=dropout

device = 'cuda'
num_trajectories = len(trajectories)
K=20
max_ep_len = max_per_game
act_dim = 1

In [44]:
max_state_token_len

97

In [110]:
def get_batch(batch_size=256, max_len=K):
        batch_inds = np.random.choice(
            np.arange(num_trajectories),
            size=batch_size,
            replace=True,
        )

        s, a, r, timesteps, mask, state_masks = [], [], [], [], [], []
        for i in range(batch_size):
            traj = trajectories[batch_inds[i]]
            si = random.randint(0, traj['rewards'].shape[0] - 1)

            # get sequences from dataset
            state_dim = max_state_token_len
            state_masks.append(traj['state_masks'][si:si + max_len].reshape(1, -1, state_dim))
            s.append(traj['states'][si:si + max_len].reshape(1, -1, max_state_token_len))
            a.append(traj['actions'][si:si + max_len].reshape(1, -1, act_dim))
            r.append(traj['rewards'][si:si + max_len].reshape(1, -1, 1))
            tlen = a[-1].shape[1]

            
            timesteps.append(np.arange(si, si + tlen).reshape(1, -1))
            timesteps[-1][timesteps[-1] >= max_ep_len] = max_ep_len-1  # padding cutoff
        
            # padding and state + reward normalization
            s[-1] = np.concatenate([np.zeros((1, max_len - tlen, state_dim)), s[-1]], axis=1)
            state_masks[-1] = np.concatenate([np.zeros((1, max_len - tlen, state_dim)), state_masks[-1]], axis=1)
            a[-1] = np.concatenate([np.ones((1, max_len - tlen, act_dim)) * 0., a[-1]], axis=1)
            r[-1] = np.concatenate([np.zeros((1, max_len - tlen, 1)), r[-1]], axis=1)
            timesteps[-1] = np.concatenate([np.zeros((1, max_len - tlen)), timesteps[-1]], axis=1)
            mask.append(np.concatenate([np.zeros((1, max_len - tlen)), np.ones((1, tlen))], axis=1))

        s = torch.from_numpy(np.concatenate(s, axis=0)).to(dtype=torch.int32, device=device)
        state_masks = torch.from_numpy(np.concatenate(state_masks, axis=0)).to(dtype=torch.int32, device=device)
        a = torch.from_numpy(np.concatenate(a, axis=0)).to(dtype=torch.int32, device=device)
        r = torch.from_numpy(np.concatenate(r, axis=0)).to(dtype=torch.float32, device=device)
        timesteps = torch.from_numpy(np.concatenate(timesteps, axis=0)).to(dtype=torch.long, device=device)
        mask = torch.from_numpy(np.concatenate(mask, axis=0)).to(device=device)

        return s, state_masks, a, r, timesteps, mask

In [46]:
s, state_masks, a, r, timesteps, mask = get_batch()

In [47]:
timesteps.shape, a.shape

(torch.Size([256, 20]), torch.Size([256, 20, 1]))

In [48]:
s[1].shape

torch.Size([20, 97])

In [49]:
len(vocab_states)

77

In [111]:
learning_rate = 4e-4
warmup_steps = 10000
loss_fn = torch.nn.CrossEntropyLoss()
weight_decay = 1e-4


In [51]:
len(vocab_states)

77

In [112]:
model = DecisionTransformer(
    state_vocab_size=len(vocab_states),
    action_vocab_size=len(vocab_actions),
    act_dim=act_dim,
    hidden_size=embed_dim,
    max_length=K,
    max_ep_len=max_ep_len,
    n_layer=n_layer,
    n_head=n_head,
    n_inner=4*embed_dim,
    activation_function=activation_function,
    n_positions=1024,
    resid_pdrop=resid_pdrop,
    attn_pdrop=attn_pdrop
)
model = model.to(device=device)

In [113]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=learning_rate,
    weight_decay=weight_decay,
)
scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer,
        lambda steps: min((steps+1)/warmup_steps, 1)
    )

In [114]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

1389711

In [117]:
def train_step(batch_size=256):
    states, state_masks, actions, rewards, timesteps, mask = get_batch(batch_size)
    action_target = torch.clone(actions)

    state_preds, action_preds, reward_preds = model.forward(
        states, actions, rewards, timesteps, state_masks, attention_mask=mask,
    )
    act_dim = action_preds.shape[2]
    action_preds = action_preds.reshape(-1, act_dim)[mask.reshape(-1) > 0]
    action_target = action_target.reshape(-1)[mask.reshape(-1) > 0]
    
    loss = loss_fn(action_preds,action_target.to(dtype=torch.long,device=device))

    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), .25)
    optimizer.step()

    # with torch.no_grad():
    #     diagnostics['training/action_error'] = torch.mean((action_preds-action_target)**2).detach().cpu().item()

    return loss.detach().cpu().item()

In [118]:
def train_iteration(num_steps, iter_num=0, print_logs=False, eval_fns=[], batch_size=256):

        train_losses = []
        logs = dict()

        train_start = time.time()

        model.train()
        for _ in range(num_steps):
            train_loss = train_step(batch_size)
            train_losses.append(train_loss)
            if scheduler is not None:
                scheduler.step()

        logs['time/training'] = time.time() - train_start

        eval_start = time.time()

        model.eval()
        for eval_fn in eval_fns:
            outputs = eval_fn(model)
            for k, v in outputs.items():
                logs[f'evaluation/{k}'] = v

        # logs['time/total'] = time.time() - self.start_time
        # logs['time/evaluation'] = time.time() - eval_start
        logs['training/train_loss_mean'] = np.mean(train_losses)
        logs['training/train_loss_std'] = np.std(train_losses)

        if print_logs:
            print('=' * 80)
            print(f'Iteration {iter_num}')
            for k, v in logs.items():
                print(f'{k}: {v}')

        return logs

In [119]:
max_iters = 10
num_steps_per_iter = 10000

In [120]:
for i in range(max_iters):
    output = train_iteration(num_steps=num_steps_per_iter, iter_num=i+1, print_logs=True,batch_size=4)

Iteration 1
time/training: 365.977614402771
training/train_loss_mean: 1.9614128202795982
training/train_loss_std: 0.586829717241452
Iteration 2
time/training: 364.17572951316833
training/train_loss_mean: 1.5557472905635834
training/train_loss_std: 0.1637498992025789
Iteration 3
time/training: 356.8068177700043
training/train_loss_mean: 1.4522001641333102
training/train_loss_std: 0.16073900625978066
Iteration 4
time/training: 348.9617145061493
training/train_loss_mean: 1.310658972465992
training/train_loss_std: 0.14532210255949657
Iteration 5
time/training: 347.64861822128296
training/train_loss_mean: 1.2273252560138703
training/train_loss_std: 0.13439001370052062
Iteration 6
time/training: 340.7488467693329
training/train_loss_mean: 1.166514534020424
training/train_loss_std: 0.1340283156591157
Iteration 7
time/training: 342.0501899719238
training/train_loss_mean: 1.1125471530526876
training/train_loss_std: 0.13609413437513065
Iteration 8
time/training: 340.1102406978607
training/train_

In [62]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

1378681

In [121]:
policies = [None,None]
wins=[0,0]
n_games = 50

In [122]:
model.eval()
model.to(device=device)

DecisionTransformer(
  (string_transformer): GPT2Model(
    (wte): Embedding(90, 128)
    (wpe): Embedding(1024, 128)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_

In [123]:
def get_move_dt(state, actions, target_return, timesteps, state_mask, move_list,move_indexes=None):
    action_probs = model.get_action(
        state,
        actions, 
        target_return, 
        timesteps, 
        state_mask
    )
    action_probs = softmax(action_probs)
    probs = []
    for move in move_list:
        probs.append(action_probs[vocab_actions[move]].item())
    probs = [prob/sum(probs) for prob in probs]
    if move_indexes is None:
        final_move = np.random.choice(move_list, p=probs)
    else:
        final_move = np.random.choice(move_indexes, p=probs)
    return final_move

In [124]:
target_return = torch.tensor(1, device=device, dtype=torch.float32).reshape(1,1)
timesteps = torch.tensor(0, device=device, dtype=torch.long).reshape(1, 1)
actions = torch.zeros((0, act_dim), device=device, dtype=torch.long)
softmax = torch.nn.Softmax()
# Simulate n_names times
for i in range(n_games):
    print("Starting game %d" % (i + 1))
    # Create game
    #game = Game([Player(get_8ed_core_gold_deck(), name="Gold"), Player(get_8ed_core_silver_deck(), name="Silver")])
    game = Game([Player(get_8ed_core_gold_deck(), name="Player1", deck_name="Gold"), Player(get_8ed_core_gold_deck(), name="Player2", deck_name="Gold")])
    game.start_game()

    game_data = {
        game.players[0].name: [],
        game.players[1].name: []
    }
    while not game.is_over():
        # Get player, valid moves
        player = game.player_with_priority
        move_indexes, move_strings = game.get_moves()
        state_string = game.get_board_string()
        state = np.array([vocab_states[word] for word in process_state(state_string)])
        state_mask = torch.ones(len(state)).reshape(1,len(state)).to(device=device,dtype=torch.long)
        state = torch.from_numpy(state).reshape(1, len(state)).to(device=device,dtype=torch.long)


        # Skip COMBAT_DAMAGE_STEP_510_1c, no choices in damage assignment
        if game.current_phase_index == Phases.COMBAT_DAMAGE_STEP_510_1c:
            assert(len(move_indexes) == 1), "Should only be 1 value move"
            game.make_move(move_indexes[0],False)
            continue

        # Get move, may have to unroll action
        if isinstance(move_strings, ActionUnroller):
            unroller = move_strings
            while not unroller.is_done():
                moves = unroller.get_legal_moves()
                if player.index == 0 or len(moves) == 1:
                    move_string = get_move(player.index,state_string, move_indexes=None, move_strings=moves)
                else:
                    state = np.array([vocab_states[word] for word in process_state(state_string)])
                    state_mask = torch.ones(len(state)).reshape(1,len(state)).to(device=device,dtype=torch.long)
                    state = torch.from_numpy(state).reshape(1, len(state)).to(device=device,dtype=torch.long)
                    
                    move_string = get_move_dt(state, actions, target_return, timesteps, state_mask, move_list=moves)
                    
                    # action_probs = model.get_action(
                    #     state,
                    #     actions, 
                    #     target_return, 
                    #     timesteps, 
                    #     state_mask
                    # )
                    # action_probs = softmax(action_probs)
                    # probs = []
                    # for move in moves:
                    #     probs.append(action_probs[vocab_actions[move]].item())
                    # probs = [prob/sum(probs) for prob in probs]
                    # move_string = np.random.choice(moves, p=probs)
                    
                    #print(move_string)
                    #import pdb; pdb.set_trace()

                # Record Move
                if len(moves) > 1:
                    game_data[player.name].append([state_string, moves, move_string])

                #print(move_string)
                state_string = unroller.register_move(move_string)


            # Apply unrolled actions to game
            unroller.make_move()
        else:
            #print(state)
            if player.index == 0 or len(move_strings) == 1:
                move = get_move(player.index, state_string, move_indexes, move_strings)
                move_string = move_strings[move_indexes.index(move)]
            else:
                state = np.array([vocab_states[word] for word in process_state(state_string)])
                state_mask = torch.ones(len(state)).reshape(1,len(state)).to(device=device,dtype=torch.long)
                state = torch.from_numpy(state).reshape(1, len(state)).to(device=device,dtype=torch.long)
                move = get_move_dt(state, actions, target_return, timesteps, state_mask, move_list=move_strings, move_indexes=move_indexes)

                # action_probs = model.get_action(
                #     state,
                #     actions, 
                #     target_return, 
                #     timesteps, 
                #     state_mask
                # )
                # action_probs = softmax(action_probs)
                # probs = []
                # for move in move_strings:
                #     probs.append(action_probs[vocab_actions[move]].item())
                # probs = [prob/sum(probs) for prob in probs]
                # move = np.random.choice(move_indexes, p=probs)

                # best_move_val = None
                # best_move_index = None
                # for i in range(len(move_strings)):
                #     move = move_strings[i]
                #     move_id = vocab_actions[move]
                #     if best_move_val is None or best_move_val < action_probs[move_id]:
                #         best_move_val = action_probs[move_id]
                #         best_move_index = i
                # move = best_move_index
                # print(move_strings[move])
                #import pdb; pdb.set_trace()

            # Record Move
            if len(move_indexes) > 1:
                game_data[player.name].append([state_string, move_strings, move_string])

            #print(move_string)
            game.make_move(move, False)

            
        #move = mcts.uct(game, itermax=5)

    if game.players[1].has_lost:
        wins[0] += 1
        reward_0 = 1
        reward_1 = -1
        #print("State", state)
    elif game.players[0].has_lost:
        wins[1] += 1
        reward_0 = -1
        reward_1 = 1
        #print("State", state)
    # Add reward info
    name_0 = game.players[0].name
    name_1 = game.players[1].name
    len_0 = len(game_data[name_0])
    len_1 = len(game_data[name_1])
    for i in range(max(len_0, len_1)):
        if i <= len_0 - 1:
            game_data[name_0][i].append(reward_0)
        if i <= len_1 - 1:
            game_data[name_1][i].append(reward_1)
    # Add game_data to overall data
    for player in game.players:
        data[player.deck_name].append(game_data[player.name])
    
    print("player 0 wins: %i, player 1 wins: %i" % (wins[0], wins[1]))


Starting game 1


/tmp/ipykernel_23963/1876890772.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  action_probs = softmax(action_probs)


player 0 wins: 1, player 1 wins: 0
Starting game 2
player 0 wins: 2, player 1 wins: 0
Starting game 3
player 0 wins: 3, player 1 wins: 0
Starting game 4
player 0 wins: 4, player 1 wins: 0
Starting game 5
player 0 wins: 5, player 1 wins: 0
Starting game 6
player 0 wins: 6, player 1 wins: 0
Starting game 7
player 0 wins: 7, player 1 wins: 0
Starting game 8
player 0 wins: 8, player 1 wins: 0
Starting game 9
player 0 wins: 9, player 1 wins: 0
Starting game 10
player 0 wins: 10, player 1 wins: 0
Starting game 11
player 0 wins: 11, player 1 wins: 0
Starting game 12
player 0 wins: 12, player 1 wins: 0
Starting game 13
player 0 wins: 13, player 1 wins: 0
Starting game 14
player 0 wins: 14, player 1 wins: 0
Starting game 15
player 0 wins: 15, player 1 wins: 0
Starting game 16
player 0 wins: 16, player 1 wins: 0
Starting game 17
player 0 wins: 17, player 1 wins: 0
Starting game 18
player 0 wins: 18, player 1 wins: 0
Starting game 19
player 0 wins: 19, player 1 wins: 0
Starting game 20
player 0 w

In [68]:
# target_return = torch.tensor(1, device=device, dtype=torch.long).reshape(1,1)
# timesteps = torch.tensor(0, device=device, dtype=torch.long).reshape(1, 1)
# actions = torch.zeros((0, act_dim), device=device, dtype=torch.float32)

# # Simulate n_names times
# for i in range(n_games):
#     print("Starting game %d" % (i + 1))
#     # Create game
#     #game = Game([Player(get_8ed_core_gold_deck(), name="Gold"), Player(get_8ed_core_silver_deck(), name="Silver")])
#     game = Game([Player(get_8ed_core_gold_deck(), name="Player1", deck_name="Gold"), Player(get_8ed_core_gold_deck(), name="Player2", deck_name="Gold")])
#     game.start_game()

#     game_data = {
#         game.players[0].name: [],
#         game.players[1].name: []
#     }
#     while not game.is_over():
#         # Get player, valid moves
#         player = game.player_with_priority
#         move_indexes, move_strings = game.get_moves()
#         state_string = game.get_board_string()
#         state = np.array([vocab_states[word] for word in process_state(state_string)])
#         state_mask = torch.ones(len(state)).reshape(1,len(state)).to(device=device,dtype=torch.long)
#         state = torch.from_numpy(state).reshape(1, len(state)).to(device=device,dtype=torch.long)


#         # Skip COMBAT_DAMAGE_STEP_510_1c, no choices in damage assignment
#         if game.current_phase_index == Phases.COMBAT_DAMAGE_STEP_510_1c:
#             assert(len(move_indexes) == 1), "Should only be 1 value move"
#             game.make_move(move_indexes[0],False)
#             continue

#         # Get move, may have to unroll action
#         if isinstance(move_strings, ActionUnroller):
#             unroller = move_strings
#             while not unroller.is_done():
#                 moves = unroller.get_legal_moves()
#                 if player.index == 0:
#                     move_string = get_move(player.index,state_string, move_indexes=None, move_strings=moves)
#                 else:
#                     state = np.array([vocab_states[word] for word in process_state(state_string)])
#                     state_mask = torch.ones(len(state)).reshape(1,len(state)).to(device=device,dtype=torch.long)
#                     state = torch.from_numpy(state).reshape(1, len(state)).to(device=device,dtype=torch.long)
#                     action_probs = model.get_action(
#                         state,
#                         actions, 
#                         target_return, 
#                         timesteps, 
#                         state_mask
#                     )
#                     best_move_string = None
#                     best_move_val = None
#                     for move in moves:
#                         move_id = vocab_actions[move]
#                         if best_move_val is None or best_move_val < action_probs[move_id]:
#                             best_move_val = action_probs[move_id]
#                             best_move_string = move
#                     move_string = best_move_string

#                 # Record Move
#                 if len(moves) > 1:
#                     game_data[player.name].append([state_string, moves, move_string])

#                 #print(move_string)
#                 state_string = unroller.register_move(move_string)
#             # Apply unrolled actions to game
#             unroller.make_move()
#         else:
#             #print(state)
#             if player.index == 0:
#                 move = get_move(player.index, state_string, move_indexes, move_strings)
#                 move_string = move_strings[move_indexes.index(move)]
#             else:
#                 state = np.array([vocab_states[word] for word in process_state(state_string)])
#                 state_mask = torch.ones(len(state)).reshape(1,len(state)).to(device=device,dtype=torch.long)
#                 state = torch.from_numpy(state).reshape(1, len(state)).to(device=device,dtype=torch.long)
#                 action_probs = model.get_action(
#                     state,
#                     actions, 
#                     target_return, 
#                     timesteps, 
#                     state_mask
#                 )
#                 best_move_string = None
#                 best_move_val = None
#                 for move in moves:
#                     move_id = vocab_actions[move]
#                     if best_move_val is None or best_move_val < action_probs[move_id]:
#                         best_move_val = action_probs[move_id]
#                         best_move_string = move
#                 move = move_indexes[move_strings.index(best_move_string)]

#             # Record Move
#             if len(move_indexes) > 1:
#                 game_data[player.name].append([state_string, move_strings, move_string])

#             #print(move_string)
#             game.make_move(int(move), False)
#         #move = mcts.uct(game, itermax=5)

#     if game.players[1].has_lost:
#         wins[0] += 1
#         reward_0 = 1
#         reward_1 = -1
#         #print("State", state)
#     elif game.players[0].has_lost:
#         wins[1] += 1
#         reward_0 = -1
#         reward_1 = 1
#         #print("State", state)
#     # Add reward info
#     name_0 = game.players[0].name
#     name_1 = game.players[1].name
#     len_0 = len(game_data[name_0])
#     len_1 = len(game_data[name_1])
#     for i in range(max(len_0, len_1)):
#         if i <= len_0 - 1:
#             game_data[name_0][i].append(reward_0)
#         if i <= len_1 - 1:
#             game_data[name_1][i].append(reward_1)
#     # Add game_data to overall data
#     for player in game.players:
#         data[player.deck_name].append(game_data[player.name])
    
# print("player 0 wins: %i, player 1 wins: %i" % (wins[0], wins[1]))


Starting game 1


ValueError: 'Red' is not in list